In [11]:
import pandas as pd
from datetime import datetime

In [51]:
imdbRaw = pd.read_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ignoredFiles/TMDB_all_movies.csv",usecols=["title", "imdb_id", "release_date", "runtime"])

In [52]:
imdbRaw

,title,release_date,runtime,imdb_id
0,Ariel,1988-10-21,73.0,tt0094675
1,Shadows in Paradise,1986-10-17,74.0,tt0092149
2,Four Rooms,1995-12-09,98.0,tt0113101
3,Judgment Night,1993-10-15,109.0,tt0107286
4,Life in Loops (A Megacities RMX),2006-01-01,80.0,tt0825671
...,...,...,...,...
1000021,Centenario,NaN,74.0,NaN
1000022,Bonafide - Messin' In Wales,NaN,0.0,NaN
1000023,Женщина с цветами и шампанским,NaN,73.0,NaN
1000024,NaN,NaN,NaN,NaN


In [59]:
# Step 1: Convert 'release_date' to datetime and extract the year
imdbRaw['release_date'] = pd.to_datetime(imdbRaw['release_date'], format='%Y-%m-%d', errors='coerce')

# Step 2: Filter rows where both 'imdb_id' and 'release_date' are valid (non-null)
valid_time_rows = imdbRaw[pd.notna(imdbRaw['imdb_id']) & pd.notna(imdbRaw['release_date'])].copy()

# Step 3: Extract the year and assign it to a new column 'release_year'
valid_time_rows['release_year'] = valid_time_rows['release_date'].dt.year

valid_rows = valid_time_rows[valid_time_rows["runtime"] >= 10]

# Now valid_rows is your "imdbProcessed" dataframe with valid rows and a 'release_year' column
imdbProcessed = valid_rows

# Optional: If you want to reset the index
imdbProcessed.reset_index(drop=True, inplace=True)

In [60]:
imdbProcessed

,title,release_date,runtime,imdb_id,release_year
0,Ariel,1988-10-21,73.0,tt0094675,1988
1,Shadows in Paradise,1986-10-17,74.0,tt0092149,1986
2,Four Rooms,1995-12-09,98.0,tt0113101,1995
3,Judgment Night,1993-10-15,109.0,tt0107286,1993
4,Life in Loops (A Megacities RMX),2006-01-01,80.0,tt0825671,2006
...,...,...,...,...,...
428147,Nikola Tesla's Secret Laboratory,2014-08-20,11.0,tt5512022,2014
428148,"Martin Mull Live from North Ridgeville, Ohio",1987-08-15,60.0,tt0484861,1987
428149,Freelance,2024-10-12,110.0,tt27634066,2024
428150,William S. Burroughs in the Dreamachine,2015-04-14,65.0,tt4595996,2015


In [61]:
imdbProcessed.to_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ians-playground/imdbProcessed.csv")